In [5]:
from pororo import Pororo

[Korean Sentence Splitter]: Initializing Kss...


In [2]:
model = Pororo(task="mt", lang="multi", model="transformer.large.multi.mtpg")

In [3]:
def cycling_translation_en(sentence):
    english = model(sentence, 'ko', 'en')
    korean = model(english, 'en', 'ko')
    return korean

In [1]:
from datasets import load_from_disk

train = load_from_disk('/opt/ml/data/train_dataset/train')
train

Dataset({
    features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
    num_rows: 3952
})

In [5]:
import pandas as pd
from tqdm.auto import tqdm
new_train = pd.DataFrame(columns=['new_question'])
new_train['new_question']=train['question']

In [13]:
new_train['new_question'][1]

'현대 인사 운영의 출발점이 어떤 책이 됐을까?'

In [14]:
for idx,question in enumerate(tqdm(train['question'])):
    new_train['new_question'][idx]=cycling_translation_en(question)


In [15]:
new_train.to_csv('/opt/ml/new_train_question.csv')

In [8]:
c=0
for question in train['question']:
    if c==5:
        break
    print(question)
    c+=1

대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?
현대적 인사조직관리의 시발점이 된 책은?
강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?
11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?
명문이 적힌 유물을 구성하는 그릇의 총 개수는?


In [13]:
train[3951]['id']

'mrc-0-003589'

In [7]:
import pandas as pd
tq = pd.read_csv("/opt/ml/new_train_question.csv")
tq['id']=0

In [10]:
tq['id'][3]

0

In [11]:
for i in range(3952):
    tq['id'][i]=train[i]['id']

/tmp/ipykernel_252/1759398614.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tq['id'][i]=train[i]['id']
/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [14]:
tq.to_csv('/opt/ml/new_train_question.csv')

## rtt한 question이 기존 question의 고유명사를 잘 살리는가?

In [6]:
ner = Pororo(task="ner", lang="ko")

In [9]:
from konlpy.tag import Mecab
from tqdm.auto import tqdm
mecab = Mecab()

In [44]:
from datasets import load_from_disk

train = load_from_disk('/opt/ml/data/train_dataset/train')
train

Dataset({
    features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
    num_rows: 3952
})

In [45]:
train[0] #title,context,question,id,answers,document_id,__index_level_0__

{'title': '미국 상원',
 'context': '미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.\\n\\n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.\\n\\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05',
 'question': '대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?',
 'id': 'mrc-1-000067',
 'answers': {'answer_start'

In [50]:
train['title'][0:4]

['미국 상원', '인사조직관리', '강희제', '금동삼존불감']

In [49]:
import pandas as pd
new_train_q = pd.read_csv("/opt/ml/new_train_question.csv")
new_train_q

,Unnamed: 0,Unnamed: 0.1,new_question,id
0,0,0,어떤 국가기관이 대통령을 포함한 미국 행정부를 견제할 권리가 있는가?,mrc-1-000067
1,1,1,현대 인사 운영의 출발점이 어떤 책이 됐을까?,mrc-0-004397
2,2,2,강희제가 누구를 위해 1717년에 썼나요?,mrc-1-000362
3,3,3,보통 11세기와 12세기에 전국의 특성이 어떤 것이냐는 것이다.,mrc-0-001510
4,4,4,그 이름으로 작성된 유물의 전체 병은 몇 개입니까?,mrc-0-000823
...,...,...,...,...
3947,3947,3947,대부분의 IO산이 형성되는 데 영향을 미치는가?,mrc-0-005285
3948,3948,3948,애니는 어디서 왔지?,mrc-1-000699
3949,3949,3949,누가 니콜리에서 1980년에 이어 상임 지휘자로 일했을까?,mrc-0-003429
3950,3950,3950,그 용암의 신화는 무엇입니까?,mrc-0-003956


In [52]:
new_train = pd.DataFrame(columns=['title','context','question','id','answers','document_id','__index_level_0__'])
new_train['question']=new_train_q['new_question']
new_train['title']=train['title']
new_train['context']=train['context']
new_train['id']=train['id']
new_train['answers']=train['answers']
new_train['document_id']=train['document_id']
new_train['__index_level_0__']=train['__index_level_0__']

In [53]:
new_train

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,어떤 국가기관이 대통령을 포함한 미국 행정부를 견제할 권리가 있는가?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293,42
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대 인사 운영의 출발점이 어떤 책이 됐을까?,mrc-0-004397,"{'answer_start': [212], 'text': ['《경영의 실제》']}",51638,2873
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 누구를 위해 1717년에 썼나요?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028,230
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",보통 11세기와 12세기에 전국의 특성이 어떤 것이냐는 것이다.,mrc-0-001510,"{'answer_start': [625], 'text': ['중국']}",34146,992
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,그 이름으로 작성된 유물의 전체 병은 몇 개입니까?,mrc-0-000823,"{'answer_start': [30], 'text': ['4개']}",47334,548
...,...,...,...,...,...,...,...
3947,이오 (위성),이오의 산\n이오의 산 목록\n 이오에는 100~150개의 산이 있다. 이들 산의 ...,대부분의 IO산이 형성되는 데 영향을 미치는가?,mrc-0-005285,"{'answer_start': [317], 'text': ['지질 구조']}",8275,3445
3948,내니 다이어리,애니의 고군분투 뉴욕 입성기!!\n\n인류학자가 꿈인 21살 소녀 '애니(스칼렛 요...,애니는 어디서 왔지?,mrc-1-000699,"{'answer_start': [616], 'text': ['뉴저지']}",37764,450
3949,빈 필하모니 관현악단,1842년에 작곡가이자 지휘자인 오토 니콜라이가 빈 궁정 오페라극장 소속 관현악단을...,누가 니콜리에서 1980년에 이어 상임 지휘자로 일했을까?,mrc-0-003429,"{'answer_start': [255], 'text': ['칼 에케르트']}",12730,2252
3950,드래곤,원어는 고대 그리스어까지 거슬러 올라간다. 영어 문헌에 이 말이 나타나기 시작한 것...,그 용암의 신화는 무엇입니까?,mrc-0-003956,"{'answer_start': [476], 'text': ['슬라브 신화']}",10924,2595


In [10]:
# 조사 및 명사 띄워주기 위해서 -> 고유명사 판별 원활히 하기 위해서!
def tokenize(text):
    return mecab.morphs(text)

In [13]:
org_questions=list(map(tokenize,train['question'])) #mecab 적용

In [27]:
org_questions[0]

['대통령',
 '을',
 '포함',
 '한',
 '미국',
 '의',
 '행정부',
 '견제',
 '권',
 '을',
 '갖',
 '는',
 '국가',
 '기관',
 '은',
 '?']

In [15]:
new_questions = list(map(tokenize,new_train['new_question'])) #mecab 적용

In [29]:
new_questions[0]

['어떤',
 '국가',
 '기관',
 '이',
 '대통령',
 '을',
 '포함',
 '한',
 '미국',
 '행정부',
 '를',
 '견제',
 '할',
 '권리',
 '가',
 '있',
 '는가',
 '?']

**mecab으로 자른 거를 띄어쓰기를 중간에 넣고 한 문장으로 만들기**

In [25]:
org_question=[]
for item in org_questions:
    k=''
    for i in item:
        k+=i+" "
    org_question.append(k.strip())

In [26]:
org_question[0]

'대통령 을 포함 한 미국 의 행정부 견제 권 을 갖 는 국가 기관 은 ?'

In [28]:
new_question=[]
for item in new_questions:
    k=''
    for i in item:
        k+=i+" "
    new_question.append(k.strip())

In [30]:
new_question[0]

'어떤 국가 기관 이 대통령 을 포함 한 미국 행정부 를 견제 할 권리 가 있 는가 ?'

**이제 pororo ner을 이용해서 기존 question 고유명사가 rtt question에도 잘 있는지 확인해본다.**

In [33]:
print(ner(org_question[0])) # 'O'판별한거 제외하고 나머지가 살아있는지 확인해보기!

[('대통령', 'CIVILIZATION'), (' ', 'O'), ('을', 'O'), (' ', 'O'), ('포함', 'O'), (' ', 'O'), ('한', 'O'), (' ', 'O'), ('미국', 'COUNTRY'), (' ', 'O'), ('의', 'O'), (' ', 'O'), ('행정부', 'ORGANIZATION'), (' ', 'O'), ('견제', 'O'), (' ', 'O'), ('권', 'O'), (' ', 'O'), ('을', 'O'), (' ', 'O'), ('갖', 'O'), (' ', 'O'), ('는', 'O'), (' ', 'O'), ('국가', 'O'), (' ', 'O'), ('기관', 'O'), (' ', 'O'), ('은', 'O'), (' ', 'O'), ('?', 'O')]


In [34]:
print(ner(new_question[0])) 

[('어떤', 'O'), (' ', 'O'), ('국가', 'O'), (' ', 'O'), ('기관', 'O'), (' ', 'O'), ('이', 'O'), (' ', 'O'), ('대통령', 'CIVILIZATION'), (' ', 'O'), ('을', 'O'), (' ', 'O'), ('포함', 'O'), (' ', 'O'), ('한', 'O'), (' ', 'O'), ('미국', 'COUNTRY'), (' ', 'O'), ('행정부', 'ORGANIZATION'), (' ', 'O'), ('를', 'O'), (' ', 'O'), ('견제', 'O'), (' ', 'O'), ('할', 'O'), (' ', 'O'), ('권리', 'O'), (' ', 'O'), ('가', 'O'), (' ', 'O'), ('있', 'O'), (' ', 'O'), ('는가', 'O'), (' ', 'O'), ('?', 'O')]


**과연 rtt한 question이 기존 question 고유명사를 잘 포함하고 있을까?**

In [40]:
matching=0
for idx in range(tqdm(len(org_question))):
    org = ner(org_question[idx])
    new = ner(new_question[idx])
    org_word=[]
    new_word=[]
    
    for org_describe in org:
        if org_describe[1]!="O":
            org_word.append(org_describe[0])
    
    for new_describe in new:
        if new_describe[1]!="O":
            new_word.append(new_describe[0])
            
    ## 새로 만든 question에 기존 question의 고유 명사가 다 들어가 있어야한다!
    
    for orgword in org_word:
        if orgword not in new_word:
            continue
    matching+=1

In [41]:
print("기존 question의 고유명사가 들어있는 rtt question 갯수는: ",matching)

기존 question의 고유명사가 들어있는 rtt question 갯수는:  3952
